In [1]:
pip install pyspark

In [2]:
from pyspark.sql import SparkSession # Import SparkSession to work with Spark DataFrames

# Create a Spark session (entry point to Spark functionality) 
spark = SparkSession.builder.appName("LondonCyclingAccident").getOrCreate() # The appName helps identify the job in Spark’s UI/logs
spark

data_path = "combined_collisions_v3.parquet" # Path to the Parquet dataset containing cycling accident records
s_df = spark.read.parquet(data_path) # Load the dataset into a Spark DataFrame

# Define the subset of columns: Geospatial Data, Accident Severity, Time, Road and Weather conditions, Locations and outcome 
selected_columns = [
    "Latitude", "Longitude", "Accident_Severity", "Date", "Day_of_Week", "Time",
    "Speed_Limit", "Weather_Conditions", "Road_Surface_Conditions",
    "Urban_or_Rural_Area", "Did_Police_Officer_Attend_Scene_of_Accident",
    "fatal", "serious", "slight", "in_london"
]

# Select only the relevant columns from the DataFrame
df_selected = s_df.select(*selected_columns)

# Define London bounding box
min_latitude, max_latitude = 51.3574645, 51.65309
min_longitude, max_longitude = -0.4483349, 0.2498128

# Filter the DataFrame to include only rows within the London bounding box
df_london = df_selected.filter(
    (df_selected["Latitude"] >= min_latitude) &
    (df_selected["Latitude"] <= max_latitude) &
    (df_selected["Longitude"] >= min_longitude) &
    (df_selected["Longitude"] <= max_longitude)
)

# Print the column names in a single line for quick inspection
print("Columns:", ", ".join(df_london.columns))

# Display the first 100 rows of the filtered DataFrame
df_london.show(100, truncate=False)

Columns: Latitude, Longitude, Accident_Severity, Date, Day_of_Week, Time, Speed_Limit, Weather_Conditions, Road_Surface_Conditions, Urban_or_Rural_Area, Did_Police_Officer_Attend_Scene_of_Accident, fatal, serious, slight, in_london
+---------+---------+-----------------+----------+-----------+-----+-----------+------------------+-----------------------+-------------------+-------------------------------------------+-----+-------+------+---------+
|Latitude |Longitude|Accident_Severity|Date      |Day_of_Week|Time |Speed_Limit|Weather_Conditions|Road_Surface_Conditions|Urban_or_Rural_Area|Did_Police_Officer_Attend_Scene_of_Accident|fatal|serious|slight|in_london|
+---------+---------+-----------------+----------+-----------+-----+-----------+------------------+-----------------------+-------------------+-------------------------------------------+-----+-------+------+---------+
|51.513039|-0.204346|3                |2005-01-24|2          |17:05|30.0       |1                 |1           

In [3]:
df_london.describe() # Show the columns extracted 

DataFrame[summary: string, Latitude: string, Longitude: string, Accident_Severity: string, Date: string, Day_of_Week: string, Time: string, Speed_Limit: string, Weather_Conditions: string, Road_Surface_Conditions: string, Urban_or_Rural_Area: string, Did_Police_Officer_Attend_Scene_of_Accident: string, fatal: string, serious: string, slight: string]

In [4]:
# Count rows in the London dataset
row_count = df_london.count()
print("Number of rows in dataset:", row_count)

Number of rows in dataset: 58456


In [5]:
df_london.toPandas().to_csv("cleaned_combined_collisions_v3.csv", index=False) # Convert the parquet file back to csv for export